In [1]:
import os
os.environ['PKG_CONFIG_PATH'] = '/ocean/projects/asc170022p/mtragoza/mambaforge/envs/4DCT/lib/pkgconfig'

In [2]:
import numpy as np
import fenics as fe
from fenics import grad, dot, dx, ds
import matplotlib.pyplot as plt

--------------------------------------------------------------------------
but there are no active ports detected (or Open MPI was unable to use
them).  This is most certainly not what you wanted.  Check your
cables, subnet manager configuration, etc.  The openib BTL will be
ignored for this job.

  Local host: dv001
--------------------------------------------------------------------------


In [3]:
n_nodes = 81
n_elements = n_nodes - 1
mesh = fe.UnitSquareMesh(n_elements, n_elements)
V = fe.FunctionSpace(mesh, 'Lagrange', 1)
mesh_spacing = 1 / n_elements
mesh_spacing

0.0125

In [4]:
# initial and Dirichlet boundary condition
ic_expression = fe.Constant(0)
u0 = fe.interpolate(ic_expression, V)
u1 = fe.interpolate(ic_expression, V)

def boundary(x, on_boundary):
    return on_boundary and fe.near(x[1], 0, 1e-14)

omega = 50 # Hz
bc_expression = fe.Expression(f'sin(2 * 3.1415 * omega * t)', degree=2, t=0, omega=omega)
bc = fe.DirichletBC(V, bc_expression, boundary)

In [5]:
# source term and Neumann boundary condition
f = fe.Expression('0', degree=0)
g = fe.Expression('0', degree=1)

In [6]:
# wave speed and time stepping parameters

# specify wave speed in meters per second (from physical constants)
rho = 1000 # kg / m^3
mu  = 4000 # Pa = N / m^2 = (kg m/s^2) / m^2 = kg / (m s^2)
c = np.sqrt(mu / rho) # (kg / (m s^2)) / (kg / m^3) = m^3 / (m s^2) = (m / s)^2

#c = 2 # meters / second

# dimensionalize the wave speed
L = 0.2 # side length in meters
c /= L # side lengths per second
print(c)

dt = 1e-4 # seconds

# check CFL condition
cfl_number = c * dt / mesh_spacing
print(cfl_number)
assert cfl_number <= 1, f'CFL condition violated ({c} x {dt} / {mesh_spacing} <= 1)'

total_time = 0.4 # seconds
n_steps = np.ceil(total_time / dt).astype(int)
n_steps

10.0
0.08


4000

In [7]:
# variational form
u = fe.TrialFunction(V)
v = fe.TestFunction(V)
a = (
    u * v * dx + 
    (c * dt)**2 * dot(grad(u), grad(v)) * dx
)
L = (
    (2 * u1 - u0 + dt**2 * f) * v * dx + 
    (g * v) * ds
)

In [8]:
# damped version

class AbsorbingBoundary(fe.UserExpression):
    def value_shape(self):
        return ()
    def eval(self, values, x):
        if x[0] < 0.05 or x[0] > 0.95 or x[1] > 0.95:
            values[0] = 0 #-1000
        else:
            values[0] = 0 #-10

k = fe.interpolate(AbsorbingBoundary(), V)

u = fe.TrialFunction(V)
v = fe.TestFunction(V)
a = (
    (1 - k * dt) * u * v * dx + 
    (c * dt)**2 * dot(grad(u), grad(v)) * dx
)
L = (
    ((2 - k * dt) * u1 - u0 + dt**2 * f) * v * dx + 
    (g * v) * ds
)

In [8]:
# get mapping from dofs to array indices
xyz = V.tabulate_dof_coordinates()
x = xyz[:,0]
y = xyz[:,1]
print(xyz.shape)

inds = np.argsort([n_nodes * x + y for x,y in xyz])
xyz[inds]

(6561, 2)


array([[0.    , 0.    ],
       [0.    , 0.0125],
       [0.    , 0.025 ],
       ...,
       [1.    , 0.975 ],
       [1.    , 0.9875],
       [1.    , 1.    ]])

In [9]:
u0.interpolate(ic_expression)
u1.interpolate(ic_expression)

u = fe.Function(V)
array = np.zeros((n_steps + 1, n_nodes, n_nodes))

t = 0
for i in range(n_steps):
    t += dt
    bc_expression.t = t
    fe.solve(a == L, u, [bc])
    u0.assign(u1)
    u1.assign(u)

    values = u.vector().get_local()
    array[i+1,...] = values[inds].reshape((n_nodes, n_nodes))

    if i % 10 == 0:
        print(f'Iteration {i+1} / {n_steps}')

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.
Solving linear variational problem.
Iteration 1 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 11 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 21 / 4000
Solving linear variational problem.
Solving linear va

Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 221 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 231 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variati

Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 441 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 451 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 461 / 4000
Solving linear variational problem.
Solving linear variational problem.
S

Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 661 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 671 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variati

Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 881 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 891 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variati

...

Solving linear variational problem.
Solving linear variational problem.
Iteration 3071 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 3081 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 3091 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem

Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 3291 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 3301 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 3311 / 4000
Solving linear variational problem.
Solving linear variational problem

Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 3511 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 3521 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear varia

Solving linear variational problem.
Solving linear variational problem.
Iteration 3721 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 3731 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 3741 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem

Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 3941 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 3951 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Iteration 3961 / 4000
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem

In [13]:
import xarray as xr
import hvplot.xarray

array = xr.DataArray(
    array,
    dims=['t', 'x', 'y'],
    coords={
        't': np.arange(0, n_steps + 1) * dt,
        'x': np.linspace(0, 1, n_nodes),
        'y': np.linspace(0, 1, n_nodes)
    }
)
array.hvplot(groupby=['t'], x='x', y='y', data_aspect=1, clim=(-2, 2), cmap='seismic', widget_type='scrubber')

Row
    [0] HoloViews(DynamicMap, sizing_mode='fixed', widget_type='scrubber')
    [1] WidgetBox(align=('end', 'start'))
        [0] Player(end=4000, width=550)

In [11]:
array.hvplot(groupby=['t', 'x'], x='y', ylim=[-2, 2], widget_location='bottom')

Column
    [0] HoloViews(DynamicMap, height=300, sizing_mode='fixed', widget_location='bottom', width=700)
    [1] WidgetBox(align=('center', 'end'))
        [0] DiscreteSlider(margin=(20, 20, 5, 20), name='t', options=OrderedDict([('0', ...]), value=0.0, width=250)
        [1] DiscreteSlider(margin=(5, 20, 20, 20), name='x', options=OrderedDict([('0', ...]), value=0.0, width=250)

In [12]:
array

<xarray.DataArray (t: 4001, x: 81, y: 81)>
array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[ 3.14098330e-02, -2.53092714e-03, -9.11498419e-06, ...,
          2.82567624e-65,  7.89622399e-66, -7.59717722e-66],
        [ 3.14098330e-02, -8.62259827e-03,  1.44734312e-03, ...,
          2.29000260e-64, -4.79464151e-65,  9.82613396e-66],
        [ 3.14098330e-02, -7.78530829e-03,  2.16482236e-03, ...,
         -5.91114387e-64,  2.93802001e-65,  1.92527457e-65],
...
        [-3.51139238e-02, -1.75964486e-01, -2.84222285e-01, ...,
         -1.36302203e-03, -1.10116749e-02, -1.45632654e-02],
        [-3.51139238e-02, -1.75724207e-01, -2.84128499e-01, ...,
         -2.12482258e-04, -9.71864164e-03, -1.32929023e-02],
        [-3.51139238e-02, -1.74842288e-01, -2.83459735e-01, ...,
          3.05806195e-04, -9.27848715e-03, -1.32382159e-02]],

       [[-3.70613512e-03, -1.38375399e-01, -2.44252225e-01, ...,
         -3.67361780e-02, -5.24129446e-02, -5.70551043e-02],
        [-3.70613512e-03, -1.37482485e-01, -2.43516265e-01, ...,
         -3.61959350e-02, -5.21074616e-02, -5.75619655e-02],
        [-3.70613512e-03, -1.37161427e-01, -2.43240350e-01, ...,
         -3.51093099e-02, -5.10102988e-02, -5.66287715e-02],
        ...,
        [-3.70613512e-03, -1.47508693e-01, -2.62841206e-01, ...,
          1.75517336e-02,  1.36768391e-02,  1.21673684e-02],
        [-3.70613512e-03, -1.47217499e-01, -2.62657178e-01, ...,
          1.86754856e-02,  1.50056679e-02,  1.35460281e-02],
        [-3.70613512e-03, -1.46272100e-01, -2.61881045e-01, ...,
          1.91382964e-02,  1.54834370e-02,  1.39085587e-02]]])
Coordinates:
  * t        (t) float64 0.0 0.0001 0.0002 0.0003 ... 0.3997 0.3998 0.3999 0.4
  * x        (x) float64 0.0 0.0125 0.025 0.0375 ... 0.9625 0.975 0.9875 1.0
  * y        (y) float64 0.0 0.0125 0.025 0.0375 ... 0.9625 0.975 0.9875 1.0

In [ ]:
%%time
import os
import subprocess

def create_video(video_name, array):
    
    print('Creating video frames...')
    os.makedirs(video_name, exist_ok=True)
    for i, frame in enumerate(array):
        frame_file = video_name + f'/frame{i:04d}.png'
        plt.imshow(frame.T, vmin=-2, vmax=2, cmap='seismic')
        plt.xlabel('x')
        plt.ylabel('y')
        plt.savefig(frame_file)
        print(frame_file)
        
    print('Creating video...')
    os.chdir(video_name)
    subprocess.call([
        'ffmpeg', '-framerate', '30', '-i', 'file%04d.png', '-r', '30', '-pix_fmt', 'yuv420p', f'{video_name}.mp4'
    ])

create_video('my_video2', array.values)

Creating video frames...
my_video2/frame0000.png
my_video2/frame0001.png
my_video2/frame0002.png
my_video2/frame0003.png
my_video2/frame0004.png
my_video2/frame0005.png
my_video2/frame0006.png
my_video2/frame0007.png
my_video2/frame0008.png
my_video2/frame0009.png
my_video2/frame0010.png
my_video2/frame0011.png
my_video2/frame0012.png
my_video2/frame0013.png
my_video2/frame0014.png
my_video2/frame0015.png
my_video2/frame0016.png
my_video2/frame0017.png
my_video2/frame0018.png
my_video2/frame0019.png
my_video2/frame0020.png
my_video2/frame0021.png
my_video2/frame0022.png
my_video2/frame0023.png
my_video2/frame0024.png
my_video2/frame0025.png
my_video2/frame0026.png
my_video2/frame0027.png
my_video2/frame0028.png
my_video2/frame0029.png
my_video2/frame0030.png
my_video2/frame0031.png
my_video2/frame0032.png
my_video2/frame0033.png
my_video2/frame0034.png
my_video2/frame0035.png
my_video2/frame0036.png
my_video2/frame0037.png
my_video2/frame0038.png
my_video2/frame0039.png
my_video2/frame

my_video2/frame0341.png
my_video2/frame0342.png
my_video2/frame0343.png
my_video2/frame0344.png
my_video2/frame0345.png
my_video2/frame0346.png
my_video2/frame0347.png
my_video2/frame0348.png
my_video2/frame0349.png
my_video2/frame0350.png
my_video2/frame0351.png
my_video2/frame0352.png
my_video2/frame0353.png
my_video2/frame0354.png
my_video2/frame0355.png
my_video2/frame0356.png
my_video2/frame0357.png
my_video2/frame0358.png
my_video2/frame0359.png
my_video2/frame0360.png
my_video2/frame0361.png
my_video2/frame0362.png
my_video2/frame0363.png
my_video2/frame0364.png
my_video2/frame0365.png
my_video2/frame0366.png
my_video2/frame0367.png
my_video2/frame0368.png
my_video2/frame0369.png
my_video2/frame0370.png
my_video2/frame0371.png
my_video2/frame0372.png
my_video2/frame0373.png
my_video2/frame0374.png
my_video2/frame0375.png
my_video2/frame0376.png
my_video2/frame0377.png
my_video2/frame0378.png
my_video2/frame0379.png
my_video2/frame0380.png
my_video2/frame0381.png
my_video2/frame0

my_video2/frame0683.png
my_video2/frame0684.png
my_video2/frame0685.png
my_video2/frame0686.png
my_video2/frame0687.png
my_video2/frame0688.png
my_video2/frame0689.png
my_video2/frame0690.png
my_video2/frame0691.png
my_video2/frame0692.png
my_video2/frame0693.png
my_video2/frame0694.png
my_video2/frame0695.png
my_video2/frame0696.png
my_video2/frame0697.png
my_video2/frame0698.png
my_video2/frame0699.png
my_video2/frame0700.png
my_video2/frame0701.png
my_video2/frame0702.png
my_video2/frame0703.png
my_video2/frame0704.png
my_video2/frame0705.png
my_video2/frame0706.png
my_video2/frame0707.png
my_video2/frame0708.png
my_video2/frame0709.png
my_video2/frame0710.png
my_video2/frame0711.png
my_video2/frame0712.png
my_video2/frame0713.png
my_video2/frame0714.png
my_video2/frame0715.png
my_video2/frame0716.png
my_video2/frame0717.png
my_video2/frame0718.png
my_video2/frame0719.png
my_video2/frame0720.png
my_video2/frame0721.png
my_video2/frame0722.png
my_video2/frame0723.png
my_video2/frame0

my_video2/frame1025.png
my_video2/frame1026.png
my_video2/frame1027.png
my_video2/frame1028.png
my_video2/frame1029.png
my_video2/frame1030.png
my_video2/frame1031.png
my_video2/frame1032.png
my_video2/frame1033.png
my_video2/frame1034.png
my_video2/frame1035.png
my_video2/frame1036.png
my_video2/frame1037.png
my_video2/frame1038.png
my_video2/frame1039.png
my_video2/frame1040.png
my_video2/frame1041.png
my_video2/frame1042.png
my_video2/frame1043.png
my_video2/frame1044.png
my_video2/frame1045.png
my_video2/frame1046.png
my_video2/frame1047.png
my_video2/frame1048.png
my_video2/frame1049.png
my_video2/frame1050.png
my_video2/frame1051.png
my_video2/frame1052.png
my_video2/frame1053.png
my_video2/frame1054.png
my_video2/frame1055.png
my_video2/frame1056.png
my_video2/frame1057.png
my_video2/frame1058.png
my_video2/frame1059.png
my_video2/frame1060.png
my_video2/frame1061.png
my_video2/frame1062.png
my_video2/frame1063.png
my_video2/frame1064.png
my_video2/frame1065.png
my_video2/frame1

my_video2/frame1367.png
my_video2/frame1368.png
my_video2/frame1369.png
my_video2/frame1370.png
my_video2/frame1371.png
my_video2/frame1372.png
my_video2/frame1373.png
my_video2/frame1374.png
my_video2/frame1375.png
my_video2/frame1376.png
my_video2/frame1377.png
my_video2/frame1378.png
my_video2/frame1379.png
my_video2/frame1380.png
my_video2/frame1381.png
my_video2/frame1382.png
my_video2/frame1383.png
my_video2/frame1384.png
my_video2/frame1385.png
my_video2/frame1386.png
my_video2/frame1387.png
my_video2/frame1388.png
my_video2/frame1389.png
my_video2/frame1390.png
my_video2/frame1391.png
my_video2/frame1392.png
my_video2/frame1393.png
my_video2/frame1394.png
my_video2/frame1395.png
my_video2/frame1396.png
my_video2/frame1397.png
my_video2/frame1398.png
my_video2/frame1399.png
my_video2/frame1400.png
my_video2/frame1401.png
my_video2/frame1402.png
my_video2/frame1403.png
my_video2/frame1404.png
my_video2/frame1405.png
my_video2/frame1406.png
my_video2/frame1407.png
my_video2/frame1

my_video2/frame1709.png
my_video2/frame1710.png
my_video2/frame1711.png
my_video2/frame1712.png
my_video2/frame1713.png
my_video2/frame1714.png
my_video2/frame1715.png
my_video2/frame1716.png
my_video2/frame1717.png
my_video2/frame1718.png
my_video2/frame1719.png
my_video2/frame1720.png
my_video2/frame1721.png
my_video2/frame1722.png
my_video2/frame1723.png
my_video2/frame1724.png
my_video2/frame1725.png
my_video2/frame1726.png
my_video2/frame1727.png
my_video2/frame1728.png
my_video2/frame1729.png
my_video2/frame1730.png
my_video2/frame1731.png
my_video2/frame1732.png
my_video2/frame1733.png
my_video2/frame1734.png
my_video2/frame1735.png
my_video2/frame1736.png
my_video2/frame1737.png
my_video2/frame1738.png
my_video2/frame1739.png
my_video2/frame1740.png
my_video2/frame1741.png
my_video2/frame1742.png
my_video2/frame1743.png
my_video2/frame1744.png
my_video2/frame1745.png
my_video2/frame1746.png
my_video2/frame1747.png
my_video2/frame1748.png
my_video2/frame1749.png
my_video2/frame1

my_video2/frame2051.png
my_video2/frame2052.png
my_video2/frame2053.png
my_video2/frame2054.png
my_video2/frame2055.png
my_video2/frame2056.png
my_video2/frame2057.png
my_video2/frame2058.png
my_video2/frame2059.png
my_video2/frame2060.png
my_video2/frame2061.png
my_video2/frame2062.png
my_video2/frame2063.png
my_video2/frame2064.png
my_video2/frame2065.png
my_video2/frame2066.png
my_video2/frame2067.png
my_video2/frame2068.png
my_video2/frame2069.png
my_video2/frame2070.png
my_video2/frame2071.png
my_video2/frame2072.png
my_video2/frame2073.png
my_video2/frame2074.png
my_video2/frame2075.png
my_video2/frame2076.png
my_video2/frame2077.png
my_video2/frame2078.png
my_video2/frame2079.png
my_video2/frame2080.png
my_video2/frame2081.png
my_video2/frame2082.png
my_video2/frame2083.png
my_video2/frame2084.png
my_video2/frame2085.png
my_video2/frame2086.png
my_video2/frame2087.png
my_video2/frame2088.png
my_video2/frame2089.png
my_video2/frame2090.png
my_video2/frame2091.png
my_video2/frame2

my_video2/frame2393.png
my_video2/frame2394.png
my_video2/frame2395.png
my_video2/frame2396.png
my_video2/frame2397.png
my_video2/frame2398.png
my_video2/frame2399.png
my_video2/frame2400.png
my_video2/frame2401.png
my_video2/frame2402.png
my_video2/frame2403.png
my_video2/frame2404.png
my_video2/frame2405.png
my_video2/frame2406.png
my_video2/frame2407.png
my_video2/frame2408.png
my_video2/frame2409.png
my_video2/frame2410.png
my_video2/frame2411.png
my_video2/frame2412.png
my_video2/frame2413.png
my_video2/frame2414.png
my_video2/frame2415.png
my_video2/frame2416.png
my_video2/frame2417.png
my_video2/frame2418.png
my_video2/frame2419.png
my_video2/frame2420.png
my_video2/frame2421.png
my_video2/frame2422.png
my_video2/frame2423.png
my_video2/frame2424.png
my_video2/frame2425.png
my_video2/frame2426.png
my_video2/frame2427.png
my_video2/frame2428.png
my_video2/frame2429.png
my_video2/frame2430.png
my_video2/frame2431.png
my_video2/frame2432.png
my_video2/frame2433.png
my_video2/frame2

my_video2/frame2735.png
my_video2/frame2736.png
my_video2/frame2737.png
my_video2/frame2738.png
my_video2/frame2739.png
my_video2/frame2740.png
my_video2/frame2741.png
my_video2/frame2742.png
my_video2/frame2743.png
my_video2/frame2744.png
my_video2/frame2745.png
my_video2/frame2746.png
my_video2/frame2747.png
my_video2/frame2748.png
my_video2/frame2749.png
my_video2/frame2750.png
my_video2/frame2751.png
my_video2/frame2752.png
my_video2/frame2753.png
my_video2/frame2754.png
my_video2/frame2755.png
my_video2/frame2756.png
my_video2/frame2757.png
my_video2/frame2758.png
my_video2/frame2759.png
my_video2/frame2760.png
my_video2/frame2761.png
my_video2/frame2762.png
my_video2/frame2763.png
my_video2/frame2764.png
my_video2/frame2765.png
my_video2/frame2766.png
my_video2/frame2767.png
my_video2/frame2768.png
my_video2/frame2769.png
my_video2/frame2770.png
my_video2/frame2771.png
my_video2/frame2772.png
my_video2/frame2773.png
my_video2/frame2774.png
my_video2/frame2775.png
my_video2/frame2

my_video2/frame3077.png
my_video2/frame3078.png
my_video2/frame3079.png
